# Question 1

### What are the 5 most listened to playlists?


SELECT playlist_id, COUNT(DISTINCT user_id) as listen_count  
FROM plays  
WHERE playlist_id <> 0  
GROUP BY playlist_id  
ORDER BY listen_count DESC  
limit 5 ;


### Given a user X, what are their 3 most listened to playlists?

There is more than one way to solve this problem, according to different definition / decision about how to measure how many time a user has listen to a playlist. 


Option 1 : to give every song in the playlist it's relative weight in the playlist, and to sum over this weights per playlist.

SELECT plays.playlist_id, SUM( 1.0 / list_count.num_tracks)  
FROM plays  
JOIN  
(SELECT playlist_id, COUNT(*) as num_tracks  
FROM listings  
GROUP BY playlist_id) as list_count  
ON list_count.playlist_id = plays.playlist_id  
WHERE user_id = X  
AND plays.playlist_id <> 0  
GROUP BY 1  
ORDER BY 2 DESC  
limit 3;


Option 2 : choose the playlist that the user spent most of his time listening to

SELECT playlist_id, SUM(listening_duration)  
FROM plays  
WHERE user_id = X  
AND playlist_id <> 0  
GROUP BY 1  
ORDER BY 2 DESC  
limit 3;

### For a user X, recommend 5 playlists they would like that they haven’t heard before

The query will retreive playlist that have many songs in common with other playlists the user has listen to, but not include in the list of the playlists the user has listen too

SELECT l.playlist_id, COUNT(DISTINCT l.track_id)  
FROM plays p, listings l  
WHERE p.user_id = X  
AND p.track_id = l.track_id  
AND p.playlist_id <> l.playlist_id  
AND l.playlist_id <> 0
AND NOT EXISTS  
(SELECT 'X'  
FROM plays up  
WHERE up.playlist_id = l.playlist_id  
AND up.user_id = p.user_id)  
GROUP BY 1  
ORDER BY 2 DESC  
limit 5;